In [ ]:
import numpy as np
import pandas as pd
import main
las_data = pd.read_csv(main.csv_las_out_full_path, delimiter=';')
wells_list = las_data['well_name'].value_counts().index.tolist()
len(wells_list)

In [ ]:
las_data_cutted = pd.DataFrame()

for well in wells_list:    
    well_las_data = las_data[las_data['well_name'] == well]
    cuted_well_las, not_empty = main.cut_data_frame_by_satur(well_las_data)
    if not_empty:
         las_data_cutted = las_data_cutted.append(cuted_well_las)

las_data_cutted.describe()

In [ ]:
# загружаем wc
water_content_data = pd.read_csv(main.data_dir + "\\water_content.csv", delimiter=';')
water_content_data = water_content_data.rename(columns={'Скважина': main.keys_dict[main.kid_well]})
water_content_data = water_content_data.rename(columns={' Начальная обводненность ': 'wc'})
water_content_data = water_content_data.rename(columns={'Дата запуска': 'date_start'})
water_content_data.head()

In [ ]:
las_data_cutted = pd.merge(las_data_cutted, water_content_data, on=main.keys_dict[main.kid_well])
las_data_cutted.describe()

In [ ]:
well_name = '531'
data_well =  las_data_cutted[las_data_cutted['well_name'] == well_name]
import scipy.optimize as optim
x_arr = data_well['DEPT'].tolist()
y_arr = data_well['kng'].tolist()

In [ ]:
# кубический сплайн
from scipy import interpolate
f_spline = interpolate.interp1d(x_arr, y_arr, kind = 'cubic')
for x, y in zip(x_arr, y_arr):
    f_calc = f_spline(x)

In [ ]:
well_name_list = las_data_cutted['well_name'].value_counts().index.tolist()
data_well = las_data_cutted[las_data_cutted['well_name'] == '554']
x_arr = data_well['DEPT'].tolist()
y_arr = data_well['kng'].tolist()
from scipy import interpolate
f_spline = interpolate.interp1d(x_arr, y_arr, kind = 'slinear') # 'cubic'

In [ ]:
from scipy import interpolate
well_name_list = las_data_cutted['well_name'].value_counts().index.tolist()
x_values = []
y_values = []
count_val = 784
for well_name in well_name_list:
    data_well =  las_data_cutted[las_data_cutted['well_name'] == well_name]
    x_arr = data_well['DEPT'].tolist()
    y_arr = data_well['kng'].tolist()
    f_spline = interpolate.interp1d(x_arr, y_arr, kind = 'slinear')
    h_start = data_well['DEPT'].min()
    h_end = data_well['DEPT'].max()
    h_step = (h_end - h_start)/count_val
    x_temp = []
    i = 0
    while(i < count_val):
        x_temp.append(float(f_spline(h_start + i * h_step)))
        i = i + 1
    x_values.append(x_temp)  
    y_values.append(data_well['wc'].tolist()[0])
print('end fill')

In [ ]:
y_values_r = [int(10 * y_val) for y_val in y_values]

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.metrics import accuracy_score
from sklearn.tree import DecisionTreeRegressor
from sklearn.cross_validation import cross_val_score

x_train, x_hold, y_train, y_hold = train_test_split(x_values, y_values_r, test_size=0.3, random_state=17)

In [ ]:
# Disable linter warnings to maintain consistency with tutorial.
# pylint: disable=invalid-name
# pylint: disable=g-bad-import-order

from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import argparse
import sys
import tempfile

from tensorflow.examples.tutorials.mnist import input_data

import tensorflow as tf

FLAGS = None


def deepnn(x):
  """deepnn builds the graph for a deep net for classifying digits.
  Args:
    x: an input tensor with the dimensions (N_examples, 784), where 784 is the
    number of pixels in a standard MNIST image.
  Returns:
    A tuple (y, keep_prob). y is a tensor of shape (N_examples, 10), with values
    equal to the logits of classifying the digit into one of 10 classes (the
    digits 0-9). keep_prob is a scalar placeholder for the probability of
    dropout.
  """
  # Reshape to use within a convolutional neural net.
  # Last dimension is for "features" - there is only one here, since images are
  # grayscale -- it would be 3 for an RGB image, 4 for RGBA, etc.
  with tf.name_scope('reshape'):
    x_image = tf.reshape(x, [-1, 28, 28, 1])

  # First convolutional layer - maps one grayscale image to 32 feature maps.
  with tf.name_scope('conv1'):
    W_conv1 = weight_variable([5, 5, 1, 32])
    b_conv1 = bias_variable([32])
    h_conv1 = tf.nn.relu(conv2d(x_image, W_conv1) + b_conv1)

  # Pooling layer - downsamples by 2X.
  with tf.name_scope('pool1'):
    h_pool1 = max_pool_2x2(h_conv1)

  # Second convolutional layer -- maps 32 feature maps to 64.
  with tf.name_scope('conv2'):
    W_conv2 = weight_variable([5, 5, 32, 64])
    b_conv2 = bias_variable([64])
    h_conv2 = tf.nn.relu(conv2d(h_pool1, W_conv2) + b_conv2)

  # Second pooling layer.
  with tf.name_scope('pool2'):
    h_pool2 = max_pool_2x2(h_conv2)

  # Fully connected layer 1 -- after 2 round of downsampling, our 28x28 image
  # is down to 7x7x64 feature maps -- maps this to 1024 features.
  with tf.name_scope('fc1'):
    W_fc1 = weight_variable([7 * 7 * 64, 1024])
    b_fc1 = bias_variable([1024])

    h_pool2_flat = tf.reshape(h_pool2, [-1, 7*7*64])
    h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat, W_fc1) + b_fc1)

  # Dropout - controls the complexity of the model, prevents co-adaptation of
  # features.
  with tf.name_scope('dropout'):
    keep_prob = tf.placeholder(tf.float32)
    h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)

  # Map the 1024 features to 10 classes, one for each digit
  with tf.name_scope('fc2'):
    W_fc2 = weight_variable([1024, 10])
    b_fc2 = bias_variable([10])

    y_conv = tf.matmul(h_fc1_drop, W_fc2) + b_fc2
  return y_conv, keep_prob


def conv2d(x, W):
  """conv2d returns a 2d convolution layer with full stride."""
  return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')


def max_pool_2x2(x):
  """max_pool_2x2 downsamples a feature map by 2X."""
  return tf.nn.max_pool(x, ksize=[1, 2, 2, 1],
                        strides=[1, 2, 2, 1], padding='SAME')


def weight_variable(shape):
  """weight_variable generates a weight variable of a given shape."""
  initial = tf.truncated_normal(shape, stddev=0.1)
  return tf.Variable(initial)


def bias_variable(shape):
  """bias_variable generates a bias variable of a given shape."""
  initial = tf.constant(0.1, shape=shape)
  return tf.Variable(initial)


In [ ]:
np.shape(x_train)[0]

In [ ]:
def create_matrix(y_values):
    y_values_matrix = []
    for val in y_values_r:
        y_arr = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
        y_arr[val] = 1
        y_values_matrix.append(y_arr)
    return y_values_matrix

In [ ]:
# формируем даннные для подачи на обучение
y_values_train_matrix = create_matrix(y_train)
y_values_hold_matrix = create_matrix(y_hold)
print(np.shape(y_values_train_matrix)[0])
np.shape(x_hold)[0]

In [ ]:
from las_deep_data_set import LasDataSet
train = LasDataSet(np.array(x_train), np.array(y_values_train_matrix), 17)

In [ ]:
train.y_values

In [ ]:
def main(_):
  print('Start load data')
  
  # Create the model
  x = tf.placeholder(tf.float32, [None, 784])

  # Define loss and optimizer
  y_ = tf.placeholder(tf.float32, [None, 10])
  
  train_count = 500 #20000

  # Build the graph for the deep net
  y_conv, keep_prob = deepnn(x)

  with tf.name_scope('loss'):
    cross_entropy = tf.nn.softmax_cross_entropy_with_logits(labels=y_,
                                                            logits=y_conv)
  cross_entropy = tf.reduce_mean(cross_entropy)

  with tf.name_scope('adam_optimizer'):
    train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)

  with tf.name_scope('accuracy'):
    correct_prediction = tf.equal(tf.argmax(y_conv, 1), tf.argmax(y_, 1))
    correct_prediction = tf.cast(correct_prediction, tf.float32)
  accuracy = tf.reduce_mean(correct_prediction)

  graph_location = tempfile.mkdtemp()
  print('Saving graph to: %s' % graph_location)
  train_writer = tf.summary.FileWriter(graph_location)
  train_writer.add_graph(tf.get_default_graph())

  with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for i in range(train_count):
      batch = train.next_batch(50)
      if i % 100 == 0:
        train_accuracy = accuracy.eval(feed_dict={
            x: batch[0], y_: batch[1], keep_prob: 1.0})
        print('step %d, training accuracy %g' % (i, train_accuracy))
      train_step.run(feed_dict={x: batch[0], y_: batch[1], keep_prob: 0.5})
    
    
    print('Calc result accuracy')
    res_accuracy = accuracy.eval(
        feed_dict={x: mnist.test.images, y_: mnist.test.labels, keep_prob: 1.0})
    print('result accuracy %g' % (res_accuracy))
    print('end calculation')

In [ ]:
tf.app.run(main=main)